In [1]:
import time
from multiprocessing import Pool
from sklearn.model_selection import ParameterGrid
import csv
from utils.PreProcess import preProcessData
from utils.TimeUtil import convertTime
from scripts.dayScript import dayScript
from classes.ParamEntry import ParamEntry

In [2]:
def movingAverageExperiment(paramList, verbose=False):
    df, dateList, paramEntry = paramList
    money = 1
    for date in dateList:
        new_df = df[(df.date == date)]
        day = dayScript(new_df, paramEntry, money, verbose)
        if verbose:
            print(date, money, day.money, day.dailyTrades)
        money = day.money
        break

    yearlyProfitPercentage = (money - 1) * 100
    paramEntry.profitPercentage = yearlyProfitPercentage
    print(paramEntry)
   
    return paramEntry

In [3]:
def bruteAnalysis(stockName, parameterDict, pool, threadPoolSize):
    csvList = []
    parameterGrid = getParameterGrid(parameterDict)
    folderName = "IntradayData_2018"
    df, dateList = preProcessData(folderName, stockName, parameterDict['shortTerm'], parameterDict['longTerm'], 'exponential')
    parameterGridSize = len(parameterGrid)
    print('param combination = ' , parameterGridSize)
    i=0
    while i<parameterGridSize:
        paramEntryList =[]
        for j in range(threadPoolSize):
            if i+j < parameterGridSize:
                params = parameterGrid[i+j]
                paramEntry = ParamEntry(params, stockName)
                paramEntryList.append([df, dateList, paramEntry])
        start = time.time()
        resultList = pool.map(movingAverageExperiment, paramEntryList)
        end = time.time()
        for paramEntry in resultList:
            csvList.append(paramEntry.getCsvPoint())
        print('Time - ',end - start)
        i+=threadPoolSize
        print(i,len(grid))
        break
    return csvList


In [4]:
# stockList = ['ACC_F1','ASHOKLEY_F1','AXISBANK_F1','BHARTIARTL_F1','RELIANCE_F1','INFY_F1','WIPRO_F1','PNB_F1','SBIN_F1','SUNPHARMA_F1','GRASIM_F1','LUPIN_F1','LT_F1','HINDUNILVR_F1']
stockList = ['LUPIN_F1', 'ASHOKLEY_F1','ACC_F1']

In [5]:
def getParameterGrid(parameterDict):
    parameterGrid = ParameterGrid(parameterDict)
    return parameterGrid

def getParameterDict():
    # # #old
#     shortTerm = [1,4,12,20]
#     longTerm = [100,200,500,1000]
#     targetPercentage = [0.01, 0.05, 0.075]
#     stopLossPercentage = [0.002, 0.006, 0.008, 0.010]
#     entryDifference = [0.00001, 0.00005, 0.00010, 0.0002]
    # # #new
    # shortTerm = [4,8,12,16]
    # longTerm = [100,200,300,400,500]
    # targetPercentage = [0.01, 0.05, 0.075]
    # stopLossPercentage = [0.006, 0.007, 0.008, 0.010]
    # entryDifference = [0.00005, 0.000075, 0.0001]
    # # # #v3
    shortTerm = [8]
    longTerm = [200]
    targetPercentage = [0.01]
    stopLossPercentage = [0.006]
    entryDifference = [0.00005]
    parameterDict = {
                    'shortTerm' : shortTerm, 
                    'longTerm': longTerm, 
                    'targetPercentage': targetPercentage, 
                    'stopLossPercentage': stopLossPercentage, 
                    'entryDifference':entryDifference
                }
    return parameterDict


In [6]:
threadPoolSize = 24
parameterDict = getParameterDict()
pool = Pool(threadPoolSize)
with open('result.csv', 'w') as f_out:
    out_colnames = ["stockName","shortTerm", "longTerm", "targetPercentage", "stopLossPercentage", "entryDifference","profitPercentage"]
    csv_writer = csv.DictWriter(f_out, fieldnames = out_colnames)
    csv_writer.writeheader()
    for stock in stockList:
        csvList = bruteAnalysis(stock[:-3], parameterDict, pool, threadPoolSize)
        for point in csvList:
            csv_writer.writerow(point)
        break
pool.terminate()
pool.join()

In [7]:
parameterDict = getParameterDict()
parameterGrid = getParameterGrid(parameterDict)
folderName = "IntradayData_2018"
stockName = 'LUPIN'
df, dateList = preProcessData(folderName, stockName, parameterDict['shortTerm'], parameterDict['longTerm'], 'exponential')


In [8]:
paramEntry = ParamEntry(parameterGrid[0],'LUPIN')

In [9]:
money = 1
verbose = True
for date in dateList:
    new_df = df[(df.date == date)]
    day = dayScript(new_df, paramEntry, money, True)
    if verbose:
        print(date, money, day.money, day.dailyTrades)
    money = day.money

# yearlyProfitPercentage = (money - 1) * 100
# paramEntry.profitPercentage = yearlyProfitPercentage
# print(paramEntry.profitPercentage)

eod
(1, 887.4, 885.8, '9:48', '15:0', -0.20026594545864107)
(20180101, 1, 0.9979973405454136, 1)
stopLoss
(1, 879.375, 874.42375, '12:39', '14:20', -0.5829293248045364)
(20180102, 0.9979973405454136, 0.9921797213866049, 1)
eod
(1, 880.325, 878.8, '13:19', '15:0', -0.19319683071593552)
(20180103, 0.9921797213866049, 0.9902628616098798, 1)
(867.0667500000001, 875.9, 876.1, 881.07995)
(867.0667500000001, 875.95, 876.5, 881.07995)
(867.0667500000001, 876.0, 876.5, 881.07995)
(867.0667500000001, 875.95, 876.4, 881.07995)
(867.0667500000001, 875.9, 876.05, 881.07995)
(867.0667500000001, 875.6, 876.25, 881.07995)
(867.0667500000001, 875.5, 876.2, 881.07995)
(867.0667500000001, 875.8, 876.4, 881.05495)
(867.0667500000001, 875.8, 876.0, 881.05495)
(867.0667500000001, 875.85, 876.0, 881.05495)
(867.0667500000001, 875.8, 876.1, 881.05495)
(867.0667500000001, 875.6, 876.0, 881.05495)
(867.0667500000001, 875.6, 875.9, 881.05495)
(867.0667500000001, 875.6, 875.85, 881.00495)
(867.0667500000001, 875.

(920.7494999999999, 930.3, 930.65, 933.7803)
(920.7494999999999, 930.1, 930.5, 933.7803)
(920.7494999999999, 930.15, 931.0, 933.7803)
(920.7494999999999, 930.3, 930.95, 933.7803)
(920.7494999999999, 930.05, 930.6, 933.7803)
(920.7494999999999, 930.1, 930.5, 933.7803)
(920.7494999999999, 929.65, 930.45, 933.7803)
(920.7494999999999, 929.7, 930.2, 933.7803)
(920.7494999999999, 929.7, 931.9, 933.7803)
(920.7494999999999, 930.7, 931.45, 933.7803)
(920.7494999999999, 930.1, 931.1, 933.7803)
(920.7494999999999, 930.3, 931.1, 933.7803)
(920.7494999999999, 928.9, 930.75, 933.7803)
(920.7494999999999, 928.4, 929.9, 933.7803)
(920.7494999999999, 928.75, 929.9, 933.7803)
(920.7494999999999, 929.0, 929.75, 933.7803)
(920.7494999999999, 928.85, 929.5, 933.7803)
(920.7494999999999, 928.0, 929.35, 933.7803)
(920.7494999999999, 928.1, 929.5, 933.7803)
(920.7494999999999, 928.4, 929.2, 933.7803)
(920.7494999999999, 928.0, 928.95, 933.7803)
(920.7494999999999, 928.0, 928.7, 933.7803)
(920.7494999999999,

(910.8494999999999, 918.4, 919.0, 921.4453)
(910.8494999999999, 918.1, 919.0, 921.4453)
(910.8494999999999, 918.2, 918.9, 921.4453)
(910.8494999999999, 918.25, 918.7, 921.4453)
(910.8494999999999, 918.2, 918.7, 921.4453)
(910.8494999999999, 918.6, 918.7, 921.4453)
(910.8494999999999, 918.05, 918.95, 921.4453)
(910.8494999999999, 917.55, 918.8, 921.4453)
(910.8494999999999, 916.45, 917.45, 921.4453)
(910.8494999999999, 916.45, 917.0, 921.4453)
(910.8494999999999, 916.5, 916.9, 921.4453)
(910.8494999999999, 916.6, 917.35, 921.4453)
(910.8494999999999, 917.15, 917.5, 921.4453)
(910.8494999999999, 917.15, 917.75, 921.4453)
(910.8494999999999, 917.6, 917.7, 921.4453)
(910.8494999999999, 917.6, 917.7, 921.4453)
(910.8494999999999, 917.5, 917.7, 921.4453)
(910.8494999999999, 917.65, 918.3, 921.4453)
(910.8494999999999, 916.8, 917.6, 921.4453)
(910.8494999999999, 916.8, 917.5, 921.4453)
(910.8494999999999, 917.3, 917.7, 921.4453)
(910.8494999999999, 917.0, 917.55, 921.4453)
(910.8494999999999,

(919.611, 928.65, 929.0, 933.9984)
(919.611, 928.6, 928.95, 933.9984)
(919.611, 928.0, 928.7, 933.9984)
(919.611, 928.35, 929.0, 933.9984)
(919.611, 928.15, 929.45, 933.9984)
(919.611, 929.0, 929.5, 933.9984)
(919.611, 928.5, 929.35, 933.9984)
(919.611, 928.2, 929.15, 933.9984)
(919.611, 928.45, 929.55, 933.9984)
(919.611, 928.55, 929.6, 933.9984)
(919.611, 928.75, 929.35, 933.9984)
(919.611, 928.2, 928.9, 933.9984)
(919.611, 928.0, 929.75, 933.9984)
(919.611, 928.2, 928.7, 933.9234)
(919.611, 928.2, 928.45, 933.9234)
(919.611, 927.35, 928.35, 933.9234)
(919.611, 927.85, 928.4, 933.6234)
(919.611, 927.5, 928.2, 933.4984)
(919.611, 927.9, 928.7, 933.3734)
(919.611, 928.0, 928.85, 933.3734)
(919.611, 928.4, 929.85, 933.3734)
(919.611, 928.9, 929.95, 933.3734)
(919.611, 929.25, 930.0, 933.3734)
(919.611, 929.3, 930.0, 933.3734)
(919.611, 929.3, 929.95, 933.3734)
(919.611, 928.75, 929.55, 933.3734)
(919.611, 928.45, 929.0, 933.3734)
(919.611, 929.0, 929.8, 933.3734)
(919.611, 929.05, 929.6

(948.8655, 959.1, 959.9, 963.5507)
(948.8655, 959.0, 959.85, 963.5507)
(948.8655, 959.05, 959.7, 963.5507)
(948.8655, 959.15, 959.95, 963.5507)
(948.8655, 959.55, 960.7, 963.5507)
(948.8655, 959.55, 960.85, 963.5507)
(948.8655, 959.95, 960.8, 963.5507)
(948.8655, 959.85, 960.5, 963.5507)
(948.8655, 959.8, 960.4, 963.5507)
(948.8655, 959.35, 960.15, 963.5507)
(948.8655, 959.05, 959.65, 963.5507)
(948.8655, 959.05, 959.7, 963.5507)
(948.8655, 958.55, 959.4, 963.5507)
(948.8655, 958.75, 959.9, 963.5507)
(948.8655, 958.55, 959.55, 963.5507)
(948.8655, 958.95, 959.5, 963.5507)
(948.8655, 958.55, 959.0, 963.5507)
(948.8655, 958.3, 958.9, 963.5507)
(948.8655, 957.2, 958.75, 963.5507)
(948.8655, 957.2, 958.8, 963.5507)
(948.8655, 957.75, 958.85, 963.5507)
(948.8655, 957.35, 958.7, 963.5507)
(948.8655, 956.0, 958.2, 963.5507)
(948.8655, 956.35, 957.9, 963.1507000000001)
(948.8655, 957.45, 959.2, 962.6507000000001)
(948.8655, 957.75, 959.25, 962.6507000000001)
(948.8655, 959.0, 959.85, 962.65070

...

(803.1375, 812.05, 813.0, 816.1175)
(803.1375, 812.05, 812.75, 816.1175)
(803.1375, 811.6, 812.5, 816.1175)
(803.1375, 811.2, 811.95, 816.1175)
(803.1375, 811.2, 811.8, 816.1175)
(803.1375, 811.7, 812.0, 816.1175)
(803.1375, 811.35, 812.0, 816.1175)
(803.1375, 811.7, 812.0, 816.1175)
(803.1375, 810.8, 811.7, 816.1175)
(803.1375, 810.8, 811.0, 816.1175)
(803.1375, 810.8, 812.0, 815.8675)
(803.1375, 811.2, 812.0, 815.8675)
(803.1375, 811.2, 812.7, 815.8675)
(803.1375, 812.1, 812.95, 815.8675)
(803.1375, 812.15, 812.95, 815.8675)
(803.1375, 812.8, 814.0, 815.8675)
(803.1375, 813.55, 814.0, 815.8675)
(803.1375, 813.45, 814.2, 815.8675)
(803.1375, 813.55, 814.0, 815.8675)
(803.1375, 813.55, 814.45, 815.8675)
(803.1375, 813.7, 814.35, 815.8675)
(803.1375, 813.95, 816.5, 815.8675)
stopLoss
(-1, 811.25, 815.8675, '12:26', '13:10', -0.5858488051062117)
(20180814, 0.9673817136107823, 0.9617143194007775, 1)
(20180816, 0.9617143194007775, 0.9617143194007775, 0)
(20180817, 0.9617143194007775, 0.961

(930.82275, 939.55, 941.0, 945.64135)
(930.82275, 939.95, 941.0, 945.64135)
(930.82275, 940.7, 941.5, 945.64135)
(930.82275, 940.0, 941.3, 945.64135)
(930.82275, 940.0, 941.05, 945.64135)
(930.82275, 940.75, 941.25, 945.64135)
(930.82275, 940.6, 942.0, 945.64135)
(930.82275, 941.65, 942.9, 945.64135)
(930.82275, 942.3, 942.9, 945.64135)
(930.82275, 942.0, 942.9, 945.64135)
(930.82275, 942.15, 942.65, 945.64135)
(930.82275, 942.0, 942.6, 945.64135)
(930.82275, 942.25, 942.85, 945.64135)
(930.82275, 942.2, 942.8, 945.64135)
(930.82275, 942.25, 942.8, 945.64135)
(930.82275, 942.55, 944.35, 945.64135)
(930.82275, 943.0, 943.95, 945.64135)
(930.82275, 943.05, 943.95, 945.64135)
(930.82275, 943.4, 944.0, 945.64135)
(930.82275, 943.65, 943.95, 945.64135)
(930.82275, 943.0, 943.9, 945.64135)
(930.82275, 943.25, 943.9, 945.64135)
(930.82275, 943.5, 943.85, 945.64135)
(930.82275, 943.2, 943.75, 945.64135)
(930.82275, 943.65, 945.1, 945.64135)
(930.82275, 944.55, 946.5, 945.64135)
stopLoss
(-1, 9

(841.7227499999999, 850.1, 852.0, 854.6763500000001)
(841.7227499999999, 851.2, 851.95, 854.6763500000001)
(841.7227499999999, 851.25, 852.0, 854.6763500000001)
(841.7227499999999, 851.3, 852.75, 854.6763500000001)
(841.7227499999999, 852.7, 854.9, 854.6763500000001)
stopLoss
(-1, 850.2249999999999, 854.6763500000001, '12:35', '12:43', -0.5407187176760092)
(842.8365, 851.2, 852.0, 856.4581000000001)
(842.8365, 851.7, 852.65, 856.4581000000001)
(842.8365, 851.75, 852.9, 856.4581000000001)
(842.8365, 851.3, 852.8, 856.4581000000001)
(842.8365, 851.0, 851.9, 856.4581000000001)
(842.8365, 850.0, 851.55, 856.4581000000001)
(842.8365, 850.0, 851.3, 856.4581000000001)
(842.8365, 850.1, 851.0, 856.0331)
(842.8365, 848.5, 850.45, 855.6081)
(842.8365, 847.4, 849.5, 854.5581000000001)
(842.8365, 846.85, 849.5, 853.3081000000001)
(842.8365, 848.0, 849.85, 853.1081)
(842.8365, 848.2, 850.0, 853.1081)
(842.8365, 848.4, 850.0, 853.1081)
(842.8365, 848.1, 848.9, 853.1081)
(842.8365, 848.05, 849.0, 853

(851.02875, 858.5, 860.8, 864.78275)
(851.02875, 858.3, 860.55, 864.78275)
(851.02875, 858.45, 860.25, 864.78275)
(851.02875, 858.2, 859.9, 864.5577499999999)
(851.02875, 859.35, 860.2, 864.5577499999999)
(851.02875, 858.5, 860.0, 864.5577499999999)
(851.02875, 859.5, 860.45, 864.5577499999999)
(851.02875, 860.0, 860.25, 864.5577499999999)
(851.02875, 859.4, 860.25, 864.5577499999999)
(851.02875, 859.5, 860.5, 864.5577499999999)
(851.02875, 860.0, 860.5, 864.5577499999999)
(851.02875, 860.25, 860.95, 864.5577499999999)
(851.02875, 860.9, 861.7, 864.5577499999999)
(851.02875, 861.0, 861.5, 864.5577499999999)
(851.02875, 861.05, 861.6, 864.5577499999999)
(851.02875, 861.35, 861.9, 864.5577499999999)
(851.02875, 861.3, 862.0, 864.5577499999999)
(851.02875, 861.5, 865.5, 864.5577499999999)
stopLoss
(-1, 859.625, 864.5577499999999, '11:8', '12:28', -0.5904377122291634)
(853.0582499999999, 861.9, 862.3, 866.84505)
(853.0582499999999, 862.25, 862.7, 866.84505)
(853.0582499999999, 862.25, 862.

(843.7275, 852.0, 853.5, 857.0385)
(843.7275, 852.35, 853.0, 857.0385)
(843.7275, 852.55, 853.1, 857.0385)
(843.7275, 852.55, 853.15, 857.0385)
(843.7275, 853.05, 853.65, 857.0385)
(843.7275, 852.25, 853.9, 857.0385)
(843.7275, 852.5, 853.05, 857.0385)
(843.7275, 851.65, 853.0, 857.0385)
(843.7275, 851.6, 852.9, 857.0385)
(843.7275, 851.7, 852.7, 857.0385)
(843.7275, 851.9, 852.35, 857.0385)
(843.7275, 851.95, 852.3, 857.0385)
(843.7275, 851.7, 852.2, 857.0385)
(843.7275, 851.9, 853.0, 857.0385)
(843.7275, 852.25, 853.0, 857.0385)
(843.7275, 852.5, 853.5, 857.0385)
(843.7275, 851.6, 852.95, 857.0385)
(843.7275, 852.0, 852.7, 857.0385)
(843.7275, 852.0, 852.7, 857.0385)
(843.7275, 852.1, 853.0, 857.0385)
(843.7275, 852.55, 853.45, 857.0385)
(843.7275, 852.75, 853.7, 857.0385)
(843.7275, 853.0, 853.9, 857.0385)
(843.7275, 853.2, 853.9, 857.0385)
(843.7275, 852.55, 854.0, 857.0385)
(843.7275, 852.85, 854.1, 857.0385)
(843.7275, 852.85, 853.9, 857.0385)
(843.7275, 852.8, 854.0, 857.0385)
(

In [11]:
money - 1

-0.06589458194666276

In [10]:
param = ParamEntry(parameterGrid[0],'a')

In [11]:
parameterGrid[0]

{'entryDifference': 5e-05,
 'longTerm': 200,
 'shortTerm': 8,
 'stopLossPercentage': 0.006,
 'targetPercentage': 0.01}

In [6]:
param.shortTerm

1